In [1]:
from pyspark import SparkContext

In [2]:
# ctrl + enter 1번만
# 원래는 분산클러스터의 master의 IP로 지정
sc = SparkContext('local')

In [3]:
dt = [(1,2),(3,4),(5,6),(1,7),(3,4)]
nRdd = sc.parallelize(dt)
nRdd.collect()

[(1, 2), (3, 4), (5, 6), (1, 7), (3, 4)]

## transform 함수

In [4]:
# 동일한 키의 값들의 합
# ex) (1,2), (1,7) => (1,9)
newRdd = nRdd.reduceByKey(lambda a,b : a+b)
newRdd.collect()

[(1, 9), (3, 8), (5, 6)]

In [5]:
newRdd1 = nRdd.flatMap(lambda v:v)
newRdd1.collect()

[1, 2, 3, 4, 5, 6, 1, 7, 3, 4]

In [10]:
# 튜플로 만들어줌, 리스트로도 만들수 있음

newRdd2 = nRdd.flatMap(lambda v:v).map(lambda v:(v,1))
newRdd2.collect()

[(1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (1, 1),
 (7, 1),
 (3, 1),
 (4, 1)]

In [11]:
# 앞부분이 키
newRdd2.reduceByKey(lambda a,b : a+b).collect()

[(1, 2), (2, 1), (3, 2), (4, 2), (5, 1), (6, 1), (7, 1)]

In [21]:
# 첫번째요소를 키로 지정하여 정렬
newRdd2.sortByKey(ascending=True).collect()

[(1, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (3, 1),
 (4, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1)]

In [20]:
newRdd2.sortByKey(ascending=False).collect()

[(7, 1),
 (6, 1),
 (5, 1),
 (4, 1),
 (4, 1),
 (3, 1),
 (3, 1),
 (2, 1),
 (1, 1),
 (1, 1)]

In [52]:
# 임의로 키를 지정하여 정렬

nRdd.sortBy(lambda v:v[1],ascending=False).collect()

[(1, 7), (5, 6), (3, 4), (3, 4), (1, 2)]

In [30]:
txtRdd = sc.textFile('data/data1/aa.txt')
txtRdd.collect()

['나는 자랑스런', '태극기 앞에', '조국과 태극기', '몸과 나는']

In [54]:
txtRdd.flatMap(lambda v:v.split()).collect()

['나는', '자랑스런', '태극기', '앞에', '조국과', '태극기', '몸과', '나는']

In [35]:
txtRdd.flatMap(lambda v:v.split()).map(lambda v:(v,1)).collect()

[('나는', 1),
 ('자랑스런', 1),
 ('태극기', 1),
 ('앞에', 1),
 ('조국과', 1),
 ('태극기', 1),
 ('몸과', 1),
 ('나는', 1)]

In [36]:
txtRdd.flatMap(lambda v:v.split()).map(lambda v:(v,1)).countByKey()

defaultdict(int, {'나는': 2, '자랑스런': 1, '태극기': 2, '앞에': 1, '조국과': 1, '몸과': 1})

In [37]:
# 각 단어의 갯수 파악
txtRdd.flatMap(lambda v:v.split()).\
    map(lambda v:(v,1)).reduceByKey(lambda a,b :a+b).collect()

[('나는', 2), ('자랑스런', 1), ('태극기', 2), ('앞에', 1), ('조국과', 1), ('몸과', 1)]

## 뉴스의 단어 개수 파악

In [38]:
newsRdd = sc.textFile('data/sport_news.txt')
newsRdd.collect()

['[스포티비뉴스=박대성 기자] 토트넘이 더는 빡빡한 주급 체계를 지킬 수 없을 전망이다. 핵심 선수를 잡으려면 더 많은 주급을 제시해야 한다. 해리 케인(28)에게 기존 주급 20만 파운드가 넘는 재계약 제안을 준비한다.',
 '',
 '영국 일간지 \'인디펜던트\'는 4일(한국시간) "토트넘이 케인과 재계약 협상을 시작했다"고 알렸다. 다른 매체 \'유로 스포츠\'도 "맨체스터 시티와 파리 생제르맹이 케인을 노리고 있다. 토트넘이 케인 붙잡기에 들어갔다"고 알렸다.',
 '',
 '케인은 토트넘 유스 출신이다. 2011년 1월 1군에 합류했고, 레이턴 오리엔트, 밀월, 노리치 시티, 레스터 시티 임대를 다녀왔다. 2013년 레스터 시티 임대 뒤에 본격적으로 토트넘에서 프로 생활을 시작했다.',
 '',
 '마우리시오 포체티노 감독 아래서 만개했다. 토트넘을 넘어 프리미어리그 대표 공격수로 발돋움했다. 조세 무리뉴 감독 아래서는 완벽한 9.5번 역할로 패스까지 눈을 떴다. 올시즌 17라운드 만에 10-10을 달성했고, 현재까지 토트넘에서 226경기 153골 35도움을 기록했다.',
 '',
 '프리미어리그 전설들도 매번 칭찬이다. 맨체스터 유나이티드 전설 리오 퍼디난드를 포함한 다수는 "웨인 루니와 티에리 앙리를 보는 것 같다. 정말 다재다능하다. 세계 최고의 9번 스트라이커"라며 엄지를 세웠다.',
 '',
 '이에 맨체스터 시티와 파리 생제르맹과 연결됐다. 맨체스터 시티는 세르히오 아구에로 빈자리와 빈공을 해결하기 위해 케인에게 접근했다. 파리 생제르맹은 킬리앙 음바페가 내년에 떠난다면 케인에게 러브콜을 보낼 전망이다. 감독도 케인을 지도했던 포체티노다.',
 '',
 '케인과 계약 기간은 2024년에 끝나지만, 토트넘은 일찍이 붙잡기에 나섰다. 현지 보도를 종합하면 기존 주급 20만 파운드(약 2억 9,000만 원) 이상을 제안할 계획이다.',
 '',
 '토트넘은 빡빡한 주급 체계를 보유하고 있었다. 레비 회장은 선수단 주급이 구단 운영에 미치는 영향

In [40]:
# 단어를 나누고 하나의 리스트로 만든 후
# 튜플 형식으로 만듬
newsRdd.flatMap(lambda v:v.split()).map(lambda v:(v,1)).collect()

[('[스포티비뉴스=박대성', 1),
 ('기자]', 1),
 ('토트넘이', 1),
 ('더는', 1),
 ('빡빡한', 1),
 ('주급', 1),
 ('체계를', 1),
 ('지킬', 1),
 ('수', 1),
 ('없을', 1),
 ('전망이다.', 1),
 ('핵심', 1),
 ('선수를', 1),
 ('잡으려면', 1),
 ('더', 1),
 ('많은', 1),
 ('주급을', 1),
 ('제시해야', 1),
 ('한다.', 1),
 ('해리', 1),
 ('케인(28)에게', 1),
 ('기존', 1),
 ('주급', 1),
 ('20만', 1),
 ('파운드가', 1),
 ('넘는', 1),
 ('재계약', 1),
 ('제안을', 1),
 ('준비한다.', 1),
 ('영국', 1),
 ('일간지', 1),
 ("'인디펜던트'는", 1),
 ('4일(한국시간)', 1),
 ('"토트넘이', 1),
 ('케인과', 1),
 ('재계약', 1),
 ('협상을', 1),
 ('시작했다"고', 1),
 ('알렸다.', 1),
 ('다른', 1),
 ('매체', 1),
 ("'유로", 1),
 ("스포츠'도", 1),
 ('"맨체스터', 1),
 ('시티와', 1),
 ('파리', 1),
 ('생제르맹이', 1),
 ('케인을', 1),
 ('노리고', 1),
 ('있다.', 1),
 ('토트넘이', 1),
 ('케인', 1),
 ('붙잡기에', 1),
 ('들어갔다"고', 1),
 ('알렸다.', 1),
 ('케인은', 1),
 ('토트넘', 1),
 ('유스', 1),
 ('출신이다.', 1),
 ('2011년', 1),
 ('1월', 1),
 ('1군에', 1),
 ('합류했고,', 1),
 ('레이턴', 1),
 ('오리엔트,', 1),
 ('밀월,', 1),
 ('노리치', 1),
 ('시티,', 1),
 ('레스터', 1),
 ('시티', 1),
 ('임대를', 1),
 ('다녀왔다.', 1),
 ('2013년', 1),
 ('레스터', 1),

In [50]:
# 각 단어의 갯수 파악
top5_news=newsRdd.flatMap(lambda v:v.split()).\
    map(lambda v:(v,1)).reduceByKey(lambda a,b :a+b).sortBy(lambda v:v[1],ascending=False,).take(5)

In [51]:
top5_news

[('주급', 5), ('20만', 5), ('케인과', 4), ('파리', 3), ('맨체스터', 3)]

## Action 함수
- transform함수로 작성 후 collect를 사용해야 리스트나 딕셔너리 형태등의로 출력됨

In [16]:
d = newRdd2.countByKey()

print(d) # 딕셔너리 형태
print(d[1]) #해당 키의 값 반환

defaultdict(<class 'int'>, {1: 2, 2: 1, 3: 2, 4: 2, 5: 1, 6: 1, 7: 1})
2
